### Ave 1h forcing from ERA to 3h and 1d to compare

In [26]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import cmocean
import glob

### Main

In [20]:
folder_ERA  = '/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/ERA5-forcing/'

In [21]:
variables = ['d2m', 'msdwlwrf', 'msdwswrf', 'msl', 'msr', 'mtpr', 't2m', 'u10', 'v10']
year = 1979

In [ ]:
for var in variables:
    for month in range(1,13):
        var_file = xr.open_dataset(f'{folder_ERA}{var}_y{year}m{month:02}.nc')
        var_mean = var_file[var].resample(time='1D').mean()
        var_mean.to_netcdf(f'{folder_ERA}{var}_1d_y{year}m{month:02}.nc')